# HitPredict Pipeline

This notebook creates a network based on a HitPredict tabular file.

## Imports Python Packages Needed in the Pipeline

In [1]:
import ndex2 # The ndex2 Python client
import itertools # convenient iteration utilities 
import requests
import json
import pandas as pd
import io
import sys
import jsonschema
import os
from datetime import datetime
import networkx as nx
sys.path.append('../../resources')
from tutorial_utils import load_tutorial_config

In [11]:
import "delim2cx"

SyntaxError: invalid syntax (<ipython-input-11-0092aab4751f>, line 1)

In [6]:
tsv.delim2cx.process_row()

AttributeError: module 'tsv' has no attribute 'delim2cx'

In [ ]:
hitpredict_path = "HitPredict-example.tsv"

## NDEx Access

Configure your NDEx connection based on server, account and password. 

See ["Using NDEx Tutorial Config Files.ipynb"] for more information.

In [2]:
# get the connection parameters from the ndex_tutorial_config.json file in your home directory.
# edit the line below to specify a different connection in the config file
my_server, my_username, my_password = load_tutorial_config("main")

# alternatively, edit and uncomment these lines to set the connection parameters manually 
# my_server = "public.ndexbio.org"
# my_username = None
# my_password = None

my_ndex=ndex2.client.Ndex2(my_server, my_username, my_password)

## Get the Load Plan:

(TODO: validate load plan with jsonschema)

In [3]:
try:
    path_to_load_plan = 'hitpredict_load_plan.json'
    load_plan = None
    with open(path_to_load_plan, 'rU') as lp:
        load_plan = json.load(lp)
        
except jsonschema.ValidationError as e1:
    print "Failed to parse the loading plan: " + e1.message
    print 'at path: ' + str(e1.absolute_path)
    print "in block: "
    print e1.instance

## Function to Create NiceCX from HitPredict Data

In [18]:
# human_tax_id = "9606"

def create_hitpredict_network(hitpredict_path, load_plan):

    df = pd.read_csv(hitpredict_path, 
                dtype=str, 
                na_filter=False, 
                delimiter='\t', 
                engine='python', 
                names=header)

    network = d2c.convert_pandas_to_nice_cx_with_load_plan(df, load_plan)
    return network
    
hitpredict_network = create_hitpredict_network(hitpredict_path)

pathway_id	pathway_name	entitya	regulator_location	typea	ida	databasea	entityb	target_location	typeb	idb	databaseb	effect	mechanism	residue	sequence	tax_id	cell_data	tissue_data	modulator_complex	target_complex	modificationa	modaseq	modificationb	modbseq	pmid	direct	notes	annotator	sentence	signor_id
SIGNOR-MM	Malignant Melanoma	KIT	receptor	protein	P10721	UNIPROT	KIT	receptor	protein	P10721	UNIPROT	up-regulates	phosphorylation	Tyr703	DHAEAALyKNLLHSK	9606									10377264	t		miannu	Identification of tyr-703 and tyr-936 as autophosphorylation sites in c-kit/scfr	SIGNOR-68643
SIGNOR-MM	Malignant Melanoma	KIT	receptor	protein	P10721	UNIPROT	KIT	receptor	protein	P10721	UNIPROT	up-regulates	phosphorylation	Tyr936	SESTNHIySNLANCS	9606									10377264	t		miannu	Identification of tyr-703 and tyr-936 as autophosphorylation sites in c-kit/scfr	SIGNOR-68647
SIGNOR-MM	Malignant Melanoma	PTEN		protein	P60484	UNIPROT	CREB1	factor	protein	P16220	UNIPROT	down-regulates activity	dephosphorylation	Ser1

## Function to Get the Description and Other Info About the Pathway

In [21]:
def add_pathway_info(network):
    url = "http://signor.uniroma2.it/getPathwayData.php?pathway=" + str(network.get_network_attribute("SIGNOR_ID"))
    print(url)
    response = requests.get(url)
    pathway_info = response.text
    dataframe = pd.read_csv(io.StringIO(pathway_info), sep='\t')
    network.set_name(dataframe.iat[0,1])
    network.set_network_attribute("Labels", [dataframe.iat[0, 0]])
    network.set_network_attribute("author", dataframe.iat[0, 3])
    network.set_network_attribute("description", dataframe.iat[0, 2])
    network.set_network_attribute("version", "0.0.1")
    network.set_network_attribute("networkType", "HitPredict"")

add_pathway_info(hitpredict_network)
print(network.get_network_attribute("Labels"))


http://signor.uniroma2.it/getPathwayData.php?pathway=SIGNOR-MM
Melanoma is a skin cancer. It might exist as distinct subtypes associated with the activation of the MAPK and the PI3K pathways even if there is an association between distinct melanoma subtypes and molecular somatic events. Mucosal, acral, and to a lesser extent, lentigo malignant melanomas, can have increased copies of CDK4, and CCND1 (cyclinD), as well as mutations in KIT receptor. NRAS is mutated in about 18% of melanomas, and seems to be more frequently activated in nodular melanomas and melanomas due to chronic sun damage. BRAF has a recurrent V600E mutation (Gain of function) in about 50–70% of melanomas, however, this mutational event is frequently reported in benign pigmented naevi, and is not fully sufficient to induce a malignant transformation. MEK1 and MEK2 are downstream from RAS and RAF, on the same MAPK pathway. Activating mutations of MEK1 and MEK2 are found in 8% of melanomas. The PI3K pathway is activated

## Set the Network Containing the Visual Style for HitPredict Networks 

In [14]:
# Use the visual properties of network ... to style each output network
# http://www.ndexbio.org/#/network/d3c5ca09-bb42-11e7-94d3-0ac135e8bacf
cytoscape_visual_properties_template_id = "f54eaef9-013c-11e8-81c8-06832d634f41"

## Function to Apply a Layout to the Network

In [8]:
def cartesian(G):
    return [{'cartesianLayout': [
        {'node': n, 'x': float(G.pos[n][0]), 'y': float(G.pos[n][1])}
        for n in G.pos
        ]}]

def apply_spring_layout(network):
    my_networkx = network.to_networkx()
    my_networkx.pos = nx.drawing.spring_layout(my_networkx)
    #my_networkx.pos = nx.drawing.circular_layout(my_networkx)
    cartesian_aspect = cartesian(my_networkx)
    network.set_opaque_aspect("cartesianCoordinates", cartesian_aspect)

## Function to Print a Sample of Nodes in the Network
#### (For a Spot-Check when Debugging)

In [9]:
def spot_check_nodes(network, attribute_name):
    for id, node in itertools.islice(network.get_nodes(), 5):
        attribute_value = my_network.get_node_attribute(node, attribute_name)
        print("%s: %s = %s" % (node.get_name(), attribute_name, attribute_value))

## Function to Upload the Network

In [10]:
def upload_network(network, server, username, password, update_uuid=False):
    if update_uuid:
        message = network.update_to(update_uuid, server, username, password)      
    else:
        message = network.upload_to(server, username, password)
    return(message)

## Function to Process One HitPredict Network

In [15]:
def process_hitpredict_network(hitpredict_path, 
                      cytoscape_visual_properties_template_id, 
                      load_plan, 
                      server, 
                      username, 
                      password):
    network = create_hitpredict_network(hitpredict_path, load_plan)
    add_pathway_info(network)
    network.apply_template(
        username=username, 
        password=password, 
        server=server, 
        uuid=cytoscape_visual_properties_template_id)
    apply_spring_layout(network)
    return upload_signor_network(network, server, username, password)

process_hitpredict_path(hitpredict_path, 
                        cytoscape_visual_properties_template_id, 
                        load_plan, 
                        my_server, 
                        my_username, 
                        my_password)

0
http://signor.uniroma2.it/getPathwayData.php?pathway=SIGNOR-MM


u'http://dev.ndexbio.org/v2/network/0c9564ac-013d-11e8-81c8-06832d634f41'

## Create a Network Set to Collect the Output Networks

In [16]:
net_set_url = my_ndex.create_networkset('HitPredict Networks' + str(datetime.now()), 
                                              'Networks from HitPredict using TSV files')
net_set_uuid = net_set_url.split('/')[-1]
print('Network set uuid: ' + net_set_uuid)

Network set uuid: 110df47f-013d-11e8-81c8-06832d634f41


## Process the HitPredict Files

In [ ]:
count = 0
limit = 3
signor_uuids = []
for pathway_id in network_id_dataframe['pathway_id']:
    upload_message = process_signor_id(
        pathway_id, 
        cytoscape_visual_properties_template_id, 
        load_plan, 
        my_server, 
        my_username, 
        my_password)
    print(upload_message)
    network_uuid = upload_message.split('/')[-1]
    signor_uuids.append(network_uuid)
    if limit:
        count += 1
        if count >= limit:
            break

#spot_check_nodes(signor_uuids[0], "TYPE")
print('Adding networks to network set')
my_ndex.add_networks_to_networkset(net_set_uuid, signor_uuids)        
print('Done')